### Process text

In [1]:
filename='testtext.txt'

In [2]:
with open(filename) as f:
    textstring=f.read()

In [3]:
import nltk.data

In [4]:
sent_detector=nltk.data.load('tokenizers/punkt/english.pickle')

In [5]:
print('\n-----\n'.join(sent_detector.tokenize(textstring.strip())))

Because of smartphone apps like Uber, taxi politics have gotten a lot of press lately.
-----
One of the claims Uber proponents make is that normal taxis do not serve everybody.
-----
In New York City, taxis are heavily regulated.
-----
There are only about 14,000 taxis allowed to pick up passengers off the street in the center of the city.
-----
Uber gets around this regulation by only picking up passengers who request a ride on the app.
-----
Because there are few taxis, critics say that they can get away with only serving the few.
-----
Some claim that taxis only serve those with the privilege to live in the city center, and that they are biased against people of color.
-----
As a New Yorker who has spent my fair share of time waiting in taxi traffic on my bicycle, I wanted to get some understanding about this issue.
-----
Are taxis actually unfair, as critics claim?


In [6]:
sent_list=sent_detector.tokenize(textstring.strip(),realign_boundaries=False)

In [7]:
import string

In [8]:
def remove_stuff(st,remove):
    import string
    return st.translate(string.maketrans('',''),remove)

In [9]:
def remove_all(st):
    return remove_stuff(st,string.digits+string.punctuation)

In [10]:
def process_string(st):
    return remove_all(st.replace('-',' ')).lower().split()

In [11]:
words_sent_list=map(process_string,sent_list)

### Get functional and frequency data

In [22]:
import numpy as np

In [36]:
from equiv_sylb import equiv_sylb
#equiv_sylb=e.equiv_sylb

In [37]:
equiv_sylb(50)

1.044970334824336

In [50]:
import pickle

In [51]:
with open('frequency_dict.pkl','rb') as f:
    freq_ser=pickle.load(f)

### Compute 

In [53]:
def get_frequency(word,ser):
    if ser.has_key(word):
        return ser[word]
    else:
        return 1


In [32]:
just_words=[item for sublist in words_sent_list for item in sublist]

In [33]:
frequency=np.array(map(lambda x: get_frequency(x,freq_ser),just_words))

In [38]:
equiv_sylb(frequency).sum()

230.34051808652225

In [39]:
total_words=len(just_words)
total_sent=len(words_sent_list)
total_sylb=equiv_sylb(frequency).sum()

In [40]:
from __future__ import division

In [45]:
def flesch_kincaid_gl(words,sent,sylb):
    return 0.39*(words/sent)+11.8*(sylb/words)-15.59

In [46]:
improved_flesch_gl=flesch_kincaid_gl(total_words,total_sent,total_sylb)
improved_flesch_gl

8.9534532897635515